<a href="https://colab.research.google.com/github/thanhdaibl1572002/data-mining-project/blob/main/DATA_MINING_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GIỚI THIỆU**